In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

import time
import random 
import requests
from bs4 import BeautifulSoup 
import os
import pickle

linkedinuser = os.environ.get('linkedinuser')
linkedpass = os.environ.get('linkedpass')



In [2]:
# Scrape a list of current user agents 
url = 'https://developers.whatismybrowser.com/useragents/explore/operating_system_name/windows/'

def get_agent_list(link,pagenum):
    """
       Use this function to scape and generate a list of windows user agent strings.
       Page num is calculated by visiting the url provided to this function and checking
       to see how many pages are availble to scrape.
    """
    try:
        for i in range(pagenum):
            time.sleep(1)
            r = requests.get(url)
            if r.status_code == 200:
                data = r.content
                soup = BeautifulSoup(data,'html.parser') 
                anchors = [a.get_text() for a in (td.find('a') for td in soup.findAll('td')) if a]
            return anchors
    except Exception as e:
        print(e)

In [3]:
def generate_user_agent(agent_list):
    try:
        num = random.randint(0,len(agent_list))
        ua = agent_list[num]
        opts = Options()
        return ua
    except Exception as e:
        print(e)



In [73]:
def login(uname,pw,agent):

    try:
        pages = []
        links = []
        offset = 25
        opts = Options()
        opts.add_argument(f"user-agent={agent}")
        driver = webdriver.Chrome('../driver/chromedriver',options=opts)
        driver.get('http://linkedin.com')
        login = driver.find_element_by_xpath('/html/body/nav/section[2]/form/div[1]/div[1]/input')
        time.sleep(random.randrange(3,8))
        login.send_keys(uname)
        password = driver.find_element_by_xpath('/html/body/nav/section[2]/form/div[1]/div[2]/input')
        time.sleep(random.randrange(3,8))
        password.send_keys(pw)   
        time.sleep(random.randrange(1,8))
        submit = driver.find_element_by_xpath('/html/body/nav/section[2]/form/div[2]/button')
        submit.send_keys(Keys.ENTER)
        time.sleep(random.randrange(3,8))
#         jobs = driver.find_element_by_css_selector('#jobs-nav-item > a')
#         jobs.send_keys(Keys.ENTER)
#         job_search = driver.find_element(By.CLASS_NAME,'jobs-search-box__text-input')
#         job_search.send_keys(job_description)  
        
    # scrape the job cards from the inital jobs landing page
        for i in range(1):
            if i == 0:
                time.sleep(random.randrange(3,8))
                driver.get('https://www.linkedin.com/jobs/search/?geoId=90000049&keywords=data%20analyst&location=Los%20Angeles%20Metropolitan%20Area')
#                 WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, "jobs-search-results__list artdeco-list")))
#                 driver.execute_script("return arguments[0].scrollIntoView(true);", copyright)
                action = ActionChains(driver)
                action.send_keys(Keys.TAB * 50)
                action.perform()
                text_contents = BeautifulSoup(driver.page_source)
                time.sleep(random.randrange(3,8))
                pages.extend(text_contents)
        for a in pages.find_all('a','job-card-search__link-wrapper js-focusable disabled ember-view', href=True):
            links.append(a['href'])
                
            else:
                driver.get(f'https://www.linkedin.com/jobs/search/?geoId=90000049&keywords=data%20analyst&location=Los%20Angeles%20Metropolitan%20Area&start={offset}')
                time.sleep(random.randrange(3,8))
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                pages.extend(BeautifulSoup(driver.page_source))
                offset += 25
                time.sleep(random.randrange(3,8))
            

        return links
    
    except Exception as e:
        print(e)
    


In [6]:
data = get_agent_list(url,11)
# 

In [7]:
ua = generate_user_agent(data)
ua

'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'

In [155]:
#this has my first scrape. Save this as a backup
# raw_html = login(uname=linkedinuser,pw=linkedpass,agent=ua)

In [74]:
test = login(uname=linkedinuser,pw=linkedpass,agent=ua)

'list' object has no attribute 'find_all'


In [32]:
test[0].find

<bound method Tag.get_text of <html class="artdeco windows" lang="en"><head>
<script async="" src="https://platform.linkedin.com/js/analytics.js"></script><script type="application/javascript">!function(i,n){void 0!==i.addEventListener&&void 0!==i.hidden&&(n.liVisibilityChangeListener=function(){i.hidden&&(n.liHasWindowHidden=!0)},i.addEventListener("visibilitychange",n.liVisibilityChangeListener))}(document,window);</script>
<title>data analyst Jobs in Los Angeles Metropolitan Area | LinkedIn</title>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="" name="description"/>
<meta content="notranslate" name="google"/>
<meta content="true" name="is-http2"/>
<meta content="voyager-web" name="service"/>
<meta content="#0077B5" name="theme-color"/>
<meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0" name="viewport"/>
<link href="/voyager/manifest.json" rel="manifest"/>
<link href="https://static-exp1.licdn.com/sc/h/br/2ch76kf69

In [69]:
tmpy = []
for a in test[0].find_all('a','job-card-search__link-wrapper js-focusable disabled ember-view', href=True):
    print(a['href']) 
    tmpy.append(a['href'])

/jobs/view/1771636252/?eBP=CwEAAAFwy-BmFCEbyeZzva4W6O56Ie5CaaC0UikHD0WWSz3ZXMCVPdBKkptb7WZhy8FigsW9V7Lrv6WhBdf2gG_-kClXQFrQZMdpCCYLWILIzkcbXddFW0wGt3h-1Fl1CJE5lHsGjdfC1EJk500XwJvZtorCkvsHPo4f4US88T6YidM4eGJAUVCKY1LSkKLbBtc9j_zFnMkyeS4uvBaejbT1IxZVXSAn9SyXf255KqvklvcJ8n8HwFcy9gW0o8ro3nSi6x2o76dvsoTAjsPPJTnTOLXauewg4NeWHu_6YYMbM8LHR5WGZWpDnJxuE9aG-D7kXB65BGIXvzBEtEnSkJGCovF-LO3F9H1Pot1Vj8qHpS3w8Vm3&refId=bea39fa7-84d0-494c-8a3e-d5d76dc4ae99&trk=d_flagship3_search_srp_jobs
/jobs/view/1769776436/?eBP=CwEAAAFwy-BmFNGmQARAS8FxRgJPTnxtJIIJDV237fYVSD09SG27gcwdTkZw5PAb4LmtnjR6XstelgMNquwpCANIlRMAXamzF22rjhtm5OsA1uSwJX_LUIHtCYC8RGaGbTsiFSmxsNxCupFlLcEXnFtrVM2RmvbLVGNYZtqkTneinMswUTFHmHC1EzZEmMGbiY9oxKKMwHElI2uHNPbD-GAEMQxiYMTtjT3i2u5nNWyJYMdzNq5Ew9J6VIXblDpHOvolchpkM6qhJg_PM6YeQQXPryM5qpbXhoePOXrjFkriPDexOWA_5PmIsUyLhokC7Sl1lHacJo51GPvjJT3BxpjjScQiJW6tT-weHa0dGVi85Xxjnon5&refId=bea39fa7-84d0-494c-8a3e-d5d76dc4ae99&trk=d_flagship3_search_srp_jobs
/jobs/view/1769776770/?eBP=CwEAAAFwy-BmFJqka5uduT3

In [71]:
tmpy[1]

'/jobs/view/1769776436/?eBP=CwEAAAFwy-BmFNGmQARAS8FxRgJPTnxtJIIJDV237fYVSD09SG27gcwdTkZw5PAb4LmtnjR6XstelgMNquwpCANIlRMAXamzF22rjhtm5OsA1uSwJX_LUIHtCYC8RGaGbTsiFSmxsNxCupFlLcEXnFtrVM2RmvbLVGNYZtqkTneinMswUTFHmHC1EzZEmMGbiY9oxKKMwHElI2uHNPbD-GAEMQxiYMTtjT3i2u5nNWyJYMdzNq5Ew9J6VIXblDpHOvolchpkM6qhJg_PM6YeQQXPryM5qpbXhoePOXrjFkriPDexOWA_5PmIsUyLhokC7Sl1lHacJo51GPvjJT3BxpjjScQiJW6tT-weHa0dGVi85Xxjnon5&refId=bea39fa7-84d0-494c-8a3e-d5d76dc4ae99&trk=d_flagship3_search_srp_jobs'

In [56]:
test[0].find('a','job-card-search__link-wrapper js-focusable disabled ember-view','href')

<a class="job-card-search__link-wrapper js-focusable disabled ember-view" data-control-id="qei+/DFFTRqLTbTvtyD68A==" href="/jobs/view/1771636252/?eBP=CwEAAAFwy-BmFCEbyeZzva4W6O56Ie5CaaC0UikHD0WWSz3ZXMCVPdBKkptb7WZhy8FigsW9V7Lrv6WhBdf2gG_-kClXQFrQZMdpCCYLWILIzkcbXddFW0wGt3h-1Fl1CJE5lHsGjdfC1EJk500XwJvZtorCkvsHPo4f4US88T6YidM4eGJAUVCKY1LSkKLbBtc9j_zFnMkyeS4uvBaejbT1IxZVXSAn9SyXf255KqvklvcJ8n8HwFcy9gW0o8ro3nSi6x2o76dvsoTAjsPPJTnTOLXauewg4NeWHu_6YYMbM8LHR5WGZWpDnJxuE9aG-D7kXB65BGIXvzBEtEnSkJGCovF-LO3F9H1Pot1Vj8qHpS3w8Vm3&amp;refId=bea39fa7-84d0-494c-8a3e-d5d76dc4ae99&amp;trk=d_flagship3_search_srp_jobs" id="ember409">            Data Analyst
              <span class="job-card-search__promoted-tag label-16dp ml1">
                Promoted
              </span>
</a>

In [ ]:
#ember4 > div.application-outlet > div.authentication-outlet > section.job-search-ext.job-search-ext--two-pane > div.jobs-search-two-pane__wrapper.jobs-search-two-pane__wrapper--two-pane > div > div > div.jobs-search-two-pane__results.display-flex > div.jobs-search-results.jobs-search-results--is-two-pane > div > ul